#สมาชิก

1.   660710746 นางสาวสุนิตา กริชทิณวงศ์
2.   660710754 นางสาวฑิฆัมพร ทองสองแก้ว
3.   660710762 นางสาวนภสร เพชรศิริ

In [1]:
!pip install duckdb

In [2]:
import duckdb

In [3]:
con = duckdb.connect(database=':memory:', read_only=False)

In [4]:
query = "select * from 'netflix_titles.csv'"

In [9]:
netflix_result = con.execute(query).fetchdf()

In [10]:
netflix_result.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [11]:
netflix_result.shape

(8807, 12)

In [12]:
netflix_result.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

#ตัวอย่าง CTE แบบหลายขั้นตอน


In [58]:
query = """
WITH
-- ขั้นที่ 1: กรองข้อมูลพื้นฐาน
basic_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL
        AND date_added IS NOT NULL
        AND release_year IS NOT NULL
),

-- ขั้นที่ 2: สรุปข้อมูลตามประเทศ
country_summary AS (
    SELECT
        country,
        COUNT(*) as total_content,
        COUNT(CASE WHEN type = 'Movie' THEN 1 END) as movies,
        COUNT(CASE WHEN type = 'TV Show' THEN 1 END) as tv_shows,
        MIN(added_year) as first_year,
        MAX(added_year) as latest_year,
        ROUND(AVG(release_year), 1) as avg_release_year,
        COUNT(DISTINCT added_year) as active_years
    FROM basic_content
    GROUP BY country
    HAVING COUNT(*) >= 10  -- เฉพาะประเทศที่มีเนื้อหาอย่างน้อย 10 เรื่อง
),

-- ขั้นที่ 3: จัดอันดับประเทศ
ranked_countries AS (
    SELECT
        *,
        RANK() OVER (ORDER BY total_content DESC) as content_rank,
        ROUND(movies * 100.0 / total_content, 1) as movie_percentage,
        ROUND(tv_shows * 100.0 / total_content, 1) as tv_percentage,
        latest_year - first_year + 1 as span_years
    FROM country_summary
)

SELECT
    content_rank as rank,
    country,
    total_content,
    movies,
    tv_shows,
    movie_percentage as "movie_%",
    tv_percentage as "tv_%",
    first_year,
    latest_year,
    span_years,
    avg_release_year
FROM ranked_countries
ORDER BY total_content DESC;
"""

# Execute และแสดงผลสรุป
result = con.execute(query).fetchdf()

print("\n สรุปเทรนด์การเพิ่มเนื้อหาตามประเทศ (Top Countries)")
display(result)


 สรุปเทรนด์การเพิ่มเนื้อหาตามประเทศ (Top Countries)


,rank,country,total_content,movies,tv_shows,movie_%,tv_%,first_year,latest_year,span_years,avg_release_year
0,1,United States,2812,2058,754,73.2,26.8,2008,2021,14,2013.3
1,2,India,972,893,79,91.9,8.1,2016,2021,6,2012.0
2,3,United Kingdom,418,206,212,49.3,50.7,2015,2021,7,2015.3
3,4,Japan,244,76,168,31.1,68.9,2015,2021,7,2014.3
4,5,South Korea,199,41,158,20.6,79.4,2016,2021,6,2017.1
5,6,Canada,181,122,59,67.4,32.6,2013,2021,9,2016.4
6,7,Spain,145,97,48,66.9,33.1,2016,2021,6,2018.0
7,8,France,124,75,49,60.5,39.5,2015,2021,7,2016.8
8,9,Mexico,110,70,40,63.6,36.4,2015,2021,7,2016.1
9,10,Egypt,106,92,14,86.8,13.2,2018,2021,4,2009.0


#การวิเคราะห์เทรนด์การเพิ่มเนื้อหาเข้า **Netflix**

In [47]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 1. ลำดับการเพิ่มเนื้อหาของประเทศ
    ROW_NUMBER() OVER (
        PARTITION BY country
        ORDER BY added_year, title
    ) as addition_sequence,

    -- 2. อันดับการผลิตในปีนั้น
    DENSE_RANK() OVER (
        PARTITION BY release_year
        ORDER BY country
    ) as yearly_country_rank,

    -- 3. จำนวนเนื้อหาสะสมของประเทศ
    COUNT(*) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as cumulative_content,

    -- 4. ค่าเฉลี่ยปีการผลิตเคลื่อนที่ 5 เรื่อง
    ROUND(AVG(release_year) OVER (
        PARTITION BY country
        ORDER BY added_year
        ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
    ), 1) as moving_avg_release_year,

    -- 5. ปีที่เพิ่มเนื้อหาก่อนหน้า
    LAG(added_year, 1) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as previous_addition_year,

    -- 6. ปีที่จะเพิ่มเนื้อหาถัดไป
    LEAD(added_year, 1) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as next_addition_year,

    -- 7. เนื้อหาแรกที่เพิ่มของประเทศ
    FIRST_VALUE(title) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as first_added_content,

    -- 8. เนื้อหาล่าสุดที่เพิ่มของประเทศ
    LAST_VALUE(title) OVER (
        PARTITION BY country
        ORDER BY added_year
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as latest_added_content,

    -- 9. อันดับการเพิ่มเนื้อหาในปีนั้นของประเทศ
    RANK() OVER (
        PARTITION BY country, added_year
        ORDER BY release_year DESC
    ) as yearly_addition_rank_by_country,

    -- 10. เปอร์เซ็นไทล์ของการเพิ่มเนื้อหาในประเทศ
    ROUND(PERCENT_RANK() OVER (
        PARTITION BY country
        ORDER BY added_year
    ), 3) as addition_percentile

FROM country_content
ORDER BY country, added_year
LIMIT 20;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,addition_sequence,yearly_country_rank,cumulative_content,moving_avg_release_year,previous_addition_year,next_addition_year,first_added_content,latest_added_content,yearly_addition_rank_by_country,addition_percentile
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,1,1,1,2014.0,<NA>,<NA>,Eyes of a Thief,Eyes of a Thief,1,0.000
1,s194,D.P.,", South Korea",TV Show,2021,2021,1,1,1,2021.0,<NA>,<NA>,D.P.,D.P.,1,0.000
2,s6553,Daemonium,Argentina,Movie,2015,2016,2,1,2,2015.0,2016,2017,Abzurdah,The Kingdom,1,0.000
3,s6083,Abzurdah,Argentina,Movie,2015,2016,1,1,2,2015.0,<NA>,2016,Abzurdah,The Kingdom,1,0.000
4,s6975,Historia de un clan,Argentina,TV Show,2015,2017,5,1,9,2015.6,2017,2017,Abzurdah,The Kingdom,7,0.036
5,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,9,1,9,2015.8,2017,2018,Abzurdah,The Kingdom,2,0.036
6,s6401,Caida del Cielo,Argentina,Movie,2016,2017,3,1,9,2015.3,2016,2017,Abzurdah,The Kingdom,2,0.036
7,s7035,Hypersomnia,Argentina,Movie,2016,2017,6,1,9,2015.8,2017,2017,Abzurdah,The Kingdom,2,0.036
8,s5115,Desire,Argentina,Movie,2017,2017,4,1,9,2015.8,2017,2017,Abzurdah,The Kingdom,1,0.036
9,s5552,The Tenth Man,Argentina,Movie,2016,2017,8,1,9,2016.0,2017,2017,Abzurdah,The Kingdom,2,0.036


#ตัวอย่างแยกย่อยในแต่ละขั้นตอน

In [38]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 1. ลำดับการเพิ่มเนื้อหาของประเทศ
    ROW_NUMBER() OVER (
        PARTITION BY country
        ORDER BY added_year, title
    ) as addition_sequence,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,addition_sequence
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,1
1,s194,D.P.,", South Korea",TV Show,2021,2021,1
2,s6553,Daemonium,Argentina,Movie,2015,2016,2
3,s6083,Abzurdah,Argentina,Movie,2015,2016,1
4,s7035,Hypersomnia,Argentina,Movie,2016,2017,6
5,s6401,Caida del Cielo,Argentina,Movie,2016,2017,3
6,s7251,La Última Fiesta,Argentina,Movie,2016,2017,7
7,s5115,Desire,Argentina,Movie,2017,2017,4
8,s6975,Historia de un clan,Argentina,TV Show,2015,2017,5
9,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,9


In [39]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 2. อันดับการผลิตในปีนั้น
    DENSE_RANK() OVER (
        PARTITION BY release_year
        ORDER BY country
    ) as yearly_country_rank,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,yearly_country_rank
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,1
1,s194,D.P.,", South Korea",TV Show,2021,2021,1
2,s6553,Daemonium,Argentina,Movie,2015,2016,1
3,s6083,Abzurdah,Argentina,Movie,2015,2016,1
4,s5115,Desire,Argentina,Movie,2017,2017,1
5,s7035,Hypersomnia,Argentina,Movie,2016,2017,1
6,s6401,Caida del Cielo,Argentina,Movie,2016,2017,1
7,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,1
8,s5552,The Tenth Man,Argentina,Movie,2016,2017,1
9,s6975,Historia de un clan,Argentina,TV Show,2015,2017,1


In [40]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 3. จำนวนเนื้อหาสะสมของประเทศ
    COUNT(*) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as cumulative_content,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,cumulative_content
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,1
1,s194,D.P.,", South Korea",TV Show,2021,2021,1
2,s6083,Abzurdah,Argentina,Movie,2015,2016,2
3,s6553,Daemonium,Argentina,Movie,2015,2016,2
4,s5552,The Tenth Man,Argentina,Movie,2016,2017,9
5,s7251,La Última Fiesta,Argentina,Movie,2016,2017,9
6,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,9
7,s7035,Hypersomnia,Argentina,Movie,2016,2017,9
8,s5115,Desire,Argentina,Movie,2017,2017,9
9,s6975,Historia de un clan,Argentina,TV Show,2015,2017,9


In [42]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 4. ค่าเฉลี่ยปีการผลิตเคลื่อนที่ 5 เรื่อง
    ROUND(AVG(release_year) OVER (
        PARTITION BY country
        ORDER BY added_year
        ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
    ), 1) as moving_avg_release_year,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,moving_avg_release_year
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,2014.0
1,s194,D.P.,", South Korea",TV Show,2021,2021,2021.0
2,s6083,Abzurdah,Argentina,Movie,2015,2016,2015.0
3,s6553,Daemonium,Argentina,Movie,2015,2016,2015.0
4,s5552,The Tenth Man,Argentina,Movie,2016,2017,2015.6
5,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,2015.3
6,s7251,La Última Fiesta,Argentina,Movie,2016,2017,2015.8
7,s7035,Hypersomnia,Argentina,Movie,2016,2017,2016.0
8,s5115,Desire,Argentina,Movie,2017,2017,2016.0
9,s6975,Historia de un clan,Argentina,TV Show,2015,2017,2015.8


In [43]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 5. ปีที่เพิ่มเนื้อหาก่อนหน้า
    LAG(added_year, 1) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as previous_addition_year,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,previous_addition_year
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,<NA>
1,s194,D.P.,", South Korea",TV Show,2021,2021,<NA>
2,s6083,Abzurdah,Argentina,Movie,2015,2016,<NA>
3,s6553,Daemonium,Argentina,Movie,2015,2016,2016
4,s5552,The Tenth Man,Argentina,Movie,2016,2017,2017
5,s7035,Hypersomnia,Argentina,Movie,2016,2017,2017
6,s7251,La Última Fiesta,Argentina,Movie,2016,2017,2017
7,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,2016
8,s6401,Caida del Cielo,Argentina,Movie,2016,2017,2017
9,s5115,Desire,Argentina,Movie,2017,2017,2017


In [33]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 6. ปีที่จะเพิ่มเนื้อหาถัดไป
    LEAD(added_year, 1) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as next_addition_year,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,next_addition_year
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,<NA>
1,s194,D.P.,", South Korea",TV Show,2021,2021,<NA>
2,s6553,Daemonium,Argentina,Movie,2015,2016,2017
3,s6083,Abzurdah,Argentina,Movie,2015,2016,2016
4,s7035,Hypersomnia,Argentina,Movie,2016,2017,2017
5,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,2017
6,s6401,Caida del Cielo,Argentina,Movie,2016,2017,2017
7,s5552,The Tenth Man,Argentina,Movie,2016,2017,2017
8,s5115,Desire,Argentina,Movie,2017,2017,2018
9,s6975,Historia de un clan,Argentina,TV Show,2015,2017,2017


In [44]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 7. เนื้อหาแรกที่เพิ่มของประเทศ
    FIRST_VALUE(title) OVER (
        PARTITION BY country
        ORDER BY added_year
    ) as first_added_content,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,first_added_content
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,Eyes of a Thief
1,s194,D.P.,", South Korea",TV Show,2021,2021,D.P.
2,s6553,Daemonium,Argentina,Movie,2015,2016,Abzurdah
3,s6083,Abzurdah,Argentina,Movie,2015,2016,Abzurdah
4,s5552,The Tenth Man,Argentina,Movie,2016,2017,Abzurdah
5,s7251,La Última Fiesta,Argentina,Movie,2016,2017,Abzurdah
6,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,Abzurdah
7,s7035,Hypersomnia,Argentina,Movie,2016,2017,Abzurdah
8,s5115,Desire,Argentina,Movie,2017,2017,Abzurdah
9,s6975,Historia de un clan,Argentina,TV Show,2015,2017,Abzurdah


In [45]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 8. เนื้อหาล่าสุดที่เพิ่มของประเทศ
    LAST_VALUE(title) OVER (
        PARTITION BY country
        ORDER BY added_year
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as latest_added_content,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,latest_added_content
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,Eyes of a Thief
1,s194,D.P.,", South Korea",TV Show,2021,2021,D.P.
2,s6083,Abzurdah,Argentina,Movie,2015,2016,Al acecho
3,s6553,Daemonium,Argentina,Movie,2015,2016,Al acecho
4,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,Al acecho
5,s6401,Caida del Cielo,Argentina,Movie,2016,2017,Al acecho
6,s5552,The Tenth Man,Argentina,Movie,2016,2017,Al acecho
7,s7035,Hypersomnia,Argentina,Movie,2016,2017,Al acecho
8,s5115,Desire,Argentina,Movie,2017,2017,Al acecho
9,s7251,La Última Fiesta,Argentina,Movie,2016,2017,Al acecho


In [36]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 9. อันดับการเพิ่มเนื้อหาในปีนั้นของประเทศ
    RANK() OVER (
        PARTITION BY country, added_year
        ORDER BY release_year DESC
    ) as yearly_addition_rank_by_country,

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,yearly_addition_rank_by_country
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,1
1,s194,D.P.,", South Korea",TV Show,2021,2021,1
2,s6553,Daemonium,Argentina,Movie,2015,2016,1
3,s6083,Abzurdah,Argentina,Movie,2015,2016,1
4,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,2
5,s5552,The Tenth Man,Argentina,Movie,2016,2017,2
6,s7035,Hypersomnia,Argentina,Movie,2016,2017,2
7,s6975,Historia de un clan,Argentina,TV Show,2015,2017,7
8,s7251,La Última Fiesta,Argentina,Movie,2016,2017,2
9,s6401,Caida del Cielo,Argentina,Movie,2016,2017,2


In [46]:
query = """
WITH country_content AS (
    SELECT
        show_id,
        title,
        country,
        type,
        release_year,
        date_added,
        CAST(SUBSTR(date_added, -4) AS INTEGER) as added_year
    FROM netflix_result
    WHERE country IS NOT NULL AND date_added IS NOT NULL
)
SELECT
    show_id,
    title,
    country,
    type,
    release_year,
    added_year,

    -- 10. เปอร์เซ็นไทล์ของการเพิ่มเนื้อหาในประเทศ
    ROUND(PERCENT_RANK() OVER (
        PARTITION BY country
        ORDER BY added_year
    ), 3) as addition_percentile

FROM country_content
ORDER BY country, added_year
LIMIT 10;
"""

country_analysis = con.execute(query).fetchdf()
display(country_analysis)

,show_id,title,country,type,release_year,added_year,addition_percentile
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,0.000
1,s194,D.P.,", South Korea",TV Show,2021,2021,0.000
2,s6553,Daemonium,Argentina,Movie,2015,2016,0.000
3,s6083,Abzurdah,Argentina,Movie,2015,2016,0.000
4,s5115,Desire,Argentina,Movie,2017,2017,0.036
5,s7035,Hypersomnia,Argentina,Movie,2016,2017,0.036
6,s5315,Todo Sobre El Asado,Argentina,Movie,2016,2017,0.036
7,s6401,Caida del Cielo,Argentina,Movie,2016,2017,0.036
8,s6975,Historia de un clan,Argentina,TV Show,2015,2017,0.036
9,s7251,La Última Fiesta,Argentina,Movie,2016,2017,0.036


In [48]:
window_analysis = con.execute(query).fetchdf()

In [49]:
window_analysis.head()

,show_id,title,country,type,release_year,added_year,addition_sequence,yearly_country_rank,cumulative_content,moving_avg_release_year,previous_addition_year,next_addition_year,first_added_content,latest_added_content,yearly_addition_rank_by_country,addition_percentile
0,s366,Eyes of a Thief,", France, Algeria",Movie,2014,2021,1,1,1,2014.0,<NA>,<NA>,Eyes of a Thief,Eyes of a Thief,1,0.000
1,s194,D.P.,", South Korea",TV Show,2021,2021,1,1,1,2021.0,<NA>,<NA>,D.P.,D.P.,1,0.000
2,s6553,Daemonium,Argentina,Movie,2015,2016,2,1,2,2015.0,2016,2017,Abzurdah,The Kingdom,1,0.000
3,s6083,Abzurdah,Argentina,Movie,2015,2016,1,1,2,2015.0,<NA>,2016,Abzurdah,The Kingdom,1,0.000
4,s6401,Caida del Cielo,Argentina,Movie,2016,2017,3,1,9,2015.3,2016,2017,Abzurdah,The Kingdom,2,0.036
